<a href="https://colab.research.google.com/github/mehrnooshkarimi/GateContrib/blob/master/combat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files

# Upload your .rar file (e.g., all_objects.rar)
uploaded = files.upload()


Saving static-1.rar to static-1.rar


In [2]:
# Install unrar utility
!apt-get install unrar -y > /dev/null

# Create extraction directory
!mkdir -p /content/objects

# Extract the RAR file
!unrar x static-1.rar /content/objects/ > /dev/null

# Confirm extraction
import os

extracted_files = os.listdir('/content/objects')
print(f"Extracted {len(extracted_files)} files:")
print(extracted_files[:5])


Extracted 30 files:
['object5.xlsx', 'object18.xlsx', 'object25.xlsx', 'object17.xlsx', 'object29.xlsx']


In [4]:
from google.colab import files

# Re-upload the scan metadata file
uploaded = files.upload()


Saving Scan_Metadata_for_ComBat.csv to Scan_Metadata_for_ComBat.csv


In [5]:
import pandas as pd
import glob

# Load the re-uploaded metadata
metadata_df = pd.read_csv("Scan_Metadata_for_ComBat.csv")

# Load all object Excel files from extracted RAR folder
file_paths = sorted(glob.glob("/content/objects/object*.xlsx"))

all_data = []

for path in file_paths:
    df = pd.read_excel(path)

    # Add object identifier from filename
    object_id = path.split("/")[-1].replace("object", "").replace(".xlsx", "")
    df['object'] = f"obj_{object_id}"

    # Merge with scan metadata
    merged = df.merge(metadata_df, on='id', how='left')
    all_data.append(merged)

# Combine all merged DataFrames
combined_df = pd.concat(all_data, ignore_index=True)

# Show result
print(f"Combined shape: {combined_df.shape}")
combined_df.head()


Combined shape: (870, 85)


,id,shape_Elongation,shape_Flatness,shape_LeastAxisLength,shape_MajorAxisLength,shape_Maximum2DDiameterColumn,shape_Maximum2DDiameterRow,shape_Maximum2DDiameterSlice,shape_Maximum3DDiameter,shape_MeshVolume,...,glszm_ZonePercentage,ngtdm_Busyness,ngtdm_Coarseness,ngtdm_Contrast,ngtdm_Strength,object,kV,mA,rotation_DDC,batch
0,2262,0.219702,0.219702,9.118732,41.505020,37.062922,37.062922,9.659195,37.092012,2330.523763,...,0.061983,3.060020,0.003819,0.037373,0.157054,obj_1,80,541,0,80_541_0
1,2263,0.254365,0.254365,9.969012,39.191836,35.123499,35.123499,10.514650,35.398802,2639.546468,...,0.040974,3.844340,0.003953,0.052148,0.132029,obj_1,100,541,0,100_541_0
2,2264,0.193572,0.191245,7.631803,39.905951,34.946475,34.946475,8.964189,34.946475,1532.394877,...,0.043145,7.062548,0.004302,0.018510,0.072837,obj_1,120,541,0,120_541_0
3,2265,0.220808,0.220808,8.653867,39.191836,34.784058,34.784058,9.201692,35.062027,1980.513753,...,0.029142,2.373093,0.004564,0.005908,0.243285,obj_1,140,462,0,140_462_0
4,2266,0.208502,0.208502,8.653867,41.505020,36.741403,36.741403,9.201692,37.004671,2097.543294,...,0.041284,5.865187,0.003882,0.025877,0.082058,obj_1,120,541,0,120_541_0


In [7]:
# Install both neuroHarmonize and neuroCombat
!pip install neuroCombat --quiet
!pip install neuroHarmonize --quiet


  Preparing metadata (setup.py) ... done


▶️ STEP 4: Apply ComBat Harmonization

In [10]:
from neuroHarmonize import harmonizationLearn, harmonizationApply

# Prepare features and batch info
features = combined_df[feature_cols].values
batch_df = pd.DataFrame({'SITE': combined_df['batch'].values})

# Learn ComBat model (fix: unpack the tuple)
model, data_std = harmonizationLearn(features, batch_df)

# Apply ComBat
harmonized_features = harmonizationApply(features, batch_df, model)

# Convert to DataFrame
harmonized_df = pd.DataFrame(harmonized_features, columns=feature_cols)
harmonized_df['id'] = combined_df['id']
harmonized_df['object'] = combined_df['object']

# Preview result
harmonized_df.head()


,shape_Elongation,shape_Flatness,shape_LeastAxisLength,shape_MajorAxisLength,shape_Maximum2DDiameterColumn,shape_Maximum2DDiameterRow,shape_Maximum2DDiameterSlice,shape_Maximum3DDiameter,shape_MeshVolume,shape_MinorAxisLength,...,glszm_SmallAreaHighGrayLevelEmphasis,glszm_SmallAreaLowGrayLevelEmphasis,glszm_ZoneEntropy,glszm_ZonePercentage,ngtdm_Busyness,ngtdm_Coarseness,ngtdm_Contrast,ngtdm_Strength,id,object
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2262,obj_1
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2263,obj_1
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2264,obj_1
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2265,obj_1
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2266,obj_1


In [11]:
# Count number of samples per batch group
combined_df['batch'].value_counts()


,count
batch,
120_336_0,330
120_541_0,120
80_541_0,90
120_92_0,60
100_541_0,30
140_462_0,30
120_92_90,30
80_541_90,30
120_92_180,30


In [12]:
# Calculate variance for each feature
feature_variances = combined_df[feature_cols].var()

# List features with zero or near-zero variance
low_var_features = feature_variances[feature_variances < 1e-6]

print(f"Low-variance features (near constant): {len(low_var_features)}")
low_var_features


Low-variance features (near constant): 0


,0


In [13]:
# Check if any NaNs are present in the features
nan_check = combined_df[feature_cols].isnull().sum()
nan_features = nan_check[nan_check > 0]

print(f"Number of features with NaNs: {len(nan_features)}")
nan_features


Number of features with NaNs: 1


,0
ngtdm_Strength,1


In [14]:
# Drop the feature with NaN
clean_feature_cols = [f for f in feature_cols if f != 'ngtdm_Strength']

# Extract cleaned data
features = combined_df[clean_feature_cols].values
batch_df = pd.DataFrame({'SITE': combined_df['batch'].values})

# Rerun ComBat
from neuroHarmonize import harmonizationLearn, harmonizationApply
model, data_std = harmonizationLearn(features, batch_df)
harmonized_features = harmonizationApply(features, batch_df, model)

# Convert to DataFrame
harmonized_df = pd.DataFrame(harmonized_features, columns=clean_feature_cols)
harmonized_df['id'] = combined_df['id']
harmonized_df['object'] = combined_df['object']

# Preview
harmonized_df.head()


,shape_Elongation,shape_Flatness,shape_LeastAxisLength,shape_MajorAxisLength,shape_Maximum2DDiameterColumn,shape_Maximum2DDiameterRow,shape_Maximum2DDiameterSlice,shape_Maximum3DDiameter,shape_MeshVolume,shape_MinorAxisLength,...,glszm_SmallAreaEmphasis,glszm_SmallAreaHighGrayLevelEmphasis,glszm_SmallAreaLowGrayLevelEmphasis,glszm_ZoneEntropy,glszm_ZonePercentage,ngtdm_Busyness,ngtdm_Coarseness,ngtdm_Contrast,id,object
0,0.224943,0.224280,9.266256,41.239041,37.157431,37.135727,9.801046,37.190219,2362.176236,9.292161,...,0.503897,16.731290,0.042563,5.025352,0.048956,5.166463,0.002896,0.031972,2262,obj_1
1,0.227812,0.230422,9.453011,41.203785,36.902406,36.879902,9.837378,37.087570,2419.263088,9.345429,...,0.552509,17.730667,0.110043,4.994073,0.037963,5.054825,0.003545,0.046726,2263,obj_1
2,0.201117,0.198646,7.973658,40.777420,35.715156,35.713324,9.296691,35.761768,1684.113441,8.075768,...,0.529161,6.696529,0.115628,3.900049,0.041141,11.005752,0.004810,0.018841,2264,obj_1
3,0.222696,0.224629,9.233115,41.180871,36.795253,36.774841,9.734154,37.075980,2325.497541,9.154038,...,0.498752,19.689897,0.021269,5.351005,0.056192,1.067151,0.004206,0.017203,2265,obj_1
4,0.215878,0.215635,8.976947,41.440180,36.576184,36.578220,9.531358,36.767941,2269.737496,8.993334,...,0.452836,9.883540,0.062370,4.754308,0.039536,8.919378,0.004499,0.026276,2266,obj_1


In [15]:
# Save to CSV
harmonized_df.to_csv("combat_harmonized_radiomics.csv", index=False)

# Download in Colab
from google.colab import files
files.download("combat_harmonized_radiomics.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

✅ Step: Recalculate ICC from Harmonized Data

In [16]:
!pip install pingouin --quiet
import pingouin as pg

# Melt the DataFrame to long format
melted = harmonized_df.melt(id_vars=['id', 'object'], var_name='feature', value_name='value')

# Calculate ICC(2,1) for each feature
icc_results = []

for feature_name, group in melted.groupby('feature'):
    try:
        icc = pg.intraclass_corr(
            data=group,
            targets='id',
            raters='object',
            ratings='value',
            nan_policy='omit'
        )
        icc2_row = icc[icc['Type'] == 'ICC2'].copy()
        icc2_row['feature'] = feature_name
        icc_results.append(icc2_row)
    except Exception as e:
        print(f"Error with {feature_name}: {e}")

# Combine results
icc_df = pd.concat(icc_results, ignore_index=True).sort_values(by='ICC', ascending=False)

# Preview top stable features
icc_df[['feature', 'ICC']].head(10)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 204.4/204.4 kB 7.7 MB/s eta 0:00:00


,feature,ICC
19,glcm_Id,0.410517
20,glcm_Idm,0.404708
47,glrlm_ShortRunEmphasis,0.386008
27,glcm_JointEntropy,0.381121
46,glrlm_RunPercentage,0.381009
17,glcm_DifferenceEntropy,0.378066
45,glrlm_RunLengthNonUniformityNormalized,0.364244
40,glrlm_GrayLevelNonUniformityNormalized,0.360592
51,glszm_GrayLevelNonUniformityNormalized,0.347131
3,firstorder_Entropy,0.345579


from matplotlib import pyplot as plt
_df_0['ICC'].plot(kind='hist', bins=20, title='ICC')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_1['ICC'].plot(kind='line', figsize=(8, 4), title='ICC')
plt.gca().spines[['top', 'right']].set_visible(False)